# Задание 1

## Задание 1.1 (5 баллов)

Реализован код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), поддерживаются следующие операции:
- сложение
- умножение
- вычитание
- унарные операции
- деление
- возведение в степень


In [ ]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
import math

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / (o_value ** 2))
            case Number():
                return Dual(self.value / float(other), self.d / float(other))

    def __rtruediv__(self, other: Number) -> "Dual":
        return Dual(float(other) / self.value, - float(other) * self.d/ self.value ** 2)

    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value ** o_value, o_value * self.value ** (o_value - 1) * self.d + self.value ** o_value * o_d * math.log(self.value))
            case Number():
                return Dual(self.value ** float(other), float(other) * self.value ** (float(other) - 1) * self.d)
    
    def __rpow__(self, other: Number) -> "Dual":
        return Dual(float(other) ** self.value, float(other) ** self.value * math.log(float(other)) * self.d)

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - float(other), self.d)

    def __rsub__(self, other: Number) -> "Dual":
        return Dual(other - self.value, self.d)
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

##### Применение

In [ ]:
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

f_diff(2)

In [ ]:
def automatic_diff(f: Callable[[float], float], x: float) -> float:
    f_diff = diff(f)
    return f_diff(x)

##### Тесты

In [ ]:
assert abs(automatic_diff(lambda x: - 12 / x ** 3 + 4 ** x, 12) - 23258159.937827) < 0.001
assert abs(automatic_diff(lambda x: 5 * x ** 2 + 2 * x + 2, 2) - 22) < 0.001
assert abs(automatic_diff(lambda x: 5 * x ** (2 * x) + 10 * x, 3) - 15308.88358) < 0.001
assert abs(automatic_diff(lambda x: - x - 5 - x ** 4, 3) + 109) < 0.001

## Задание 1.2 (7 баллов)
Реализована поддержка функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

In [ ]:
def exp(obj: Union["Dual", Number]) -> Union["Dual", Number]:
    match obj:
        case Dual(o_value, o_d):
            return Dual(math.exp(o_value), math.exp(o_value) * o_d)
        case Number():
            return math.exp(float(obj))

def log(obj: Union["Dual", Number]) -> Union["Dual", Number]:
    match obj:
        case Dual(o_value, o_d):
            return Dual(math.log(o_value), o_d / o_value)
        case Number():
            return math.log(float(obj))

def cos(obj: Union["Dual", Number]) -> Union["Dual", Number]:
    match obj:
        case Dual(o_value, o_d):
            return Dual(math.cos(o_value), - math.sin(o_value) * o_d)
        case Number():
            return math.cos(float(obj))

def sin(obj: Union["Dual", Number]) -> Union["Dual", Number]:
    match obj:
        case Dual(o_value, o_d):
            return Dual(math.sin(o_value), math.cos(o_value) * o_d)
        case Number():
            return math.sin(float(obj))

##### Тесты

In [ ]:
assert abs(automatic_diff(lambda x: cos(x ** 2 + 2 / x), 3) - 1.383990) < 0.001
assert abs(automatic_diff(lambda x: 2 * sin(x ** 2) + log(3 * x) ** 2, 10) - 35.172994) < 0.001
assert abs(automatic_diff(lambda x: -x * sin(20 / x), 11) + 1.414774) < 0.001
assert abs(automatic_diff(lambda x: exp(3) * exp(x), 2) - 148.413159) < 0.001

## Задание 1.3 (3 балла)

Реализуем функцию **численного** дифференцирования (+5 баллов) и сравним ее с функцией `derivative` из библиотеки `scipy`.

In [ ]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

In [ ]:
def numerical_diff(f: Callable[[float], float], x: float, h: float = 0.1) -> float:
    return (f(x+h) - f(x-h)) / (2 * h)

In [ ]:
derivative(f, 2)

In [ ]:
numerical_diff(f, 2)

Сравним результаты **численного** и **автоматического** дифференцирования.

In [ ]:
from typing import Any

def compare(f: Callable[[float], float], x: float, eps: float = 0.001, **kwargs: Any):
    assert abs(automatic_diff(f, x) - numerical_diff(f, x, **kwargs)) < eps, "Too big difference"

In [ ]:
compare(lambda x: - 12 / x ** 3 + 4 ** x, 10, eps = 1, h = 0.0000001)
compare(lambda x: 5 * x ** 2 + 2 * x + 2, 2)
compare(lambda x: 5 * x ** (2 * x) + 10 * x, 3, h = 0.0001)
compare(lambda x: - x - 5 - x ** 4, 3, h = 0.001)
compare(lambda x: exp(3) * exp(x), 2, h = 0.001)
compare(lambda x: cos(x ** 2 + 2 / x), 3, h = 0.01)
compare(lambda x: 2 * sin(x ** 2) + log(3 * x) ** 2, 10, h = 0.0001)
compare(lambda x: -x * sin(20 / x), 11)

## Задание 1.5 (7 баллов)

Реализована поддержка функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [ ]:
def diff(f: Callable[[Any], float]) -> Callable[[Any], float]:
    def _new_f(*args):
        diff_for_arg = []
        for i, arg in enumerate(args):
            new_args = args[0:i] + (Dual(arg, 1),) + args[i+1:]
            diff_for_arg.append(f(*new_args).d)
        return diff_for_arg
    return _new_f

In [ ]:
@diff
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  

f(10, 10, 10)